In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

def processar_planilhas():
    recebimento = pd.read_excel('recebimento_ajustado.xlsx')
    expedicao = pd.read_excel('expedicao.xlsx')
    lista_produtos = pd.read_excel('listadeprodutos.xlsx')
    matriz_distancias = pd.read_excel('matriz_distancias.xlsx', index_col=0)

    expedicao = expedicao.rename(columns={
        'Nº da ordem de transporte': 'ordem',
        'Material': 'material',
        'Qtd.teórica origem': 'quantidade',
        'Posição dep.origem': 'origem',
        'Posição de destino enclausuramentos': 'destino',
        'Data de criação': 'data',
        'Hora da criação': 'hora'
    })

    recebimento = recebimento.rename(columns={
        'Nº da ordem de transporte': 'ordem',
        'tipo de produto': 'material',
        'Qtd.teórica origem': 'quantidade',
        'posição de origem ': 'origem',
        'Posição dep.destino': 'destino',
        'Data de criação': 'data',
        'Hora da criação': 'hora'
    })

    def padronizar_localizacao(loc):
      if pd.isna(loc):
          return loc
      loc = str(loc)

      if loc.isdigit():
          return f"e{loc.zfill(2)}"

      elif re.match(r'^A[VX][A-Za-z0-9]+$', loc):
          return loc.upper().strip()

      elif loc.startswith(('Esteira', 'E')):
          return loc

      else:
          return loc.upper().strip()

    expedicao['destino'] = expedicao['destino'].apply(padronizar_localizacao)
    recebimento['origem'] = recebimento['origem'].apply(padronizar_localizacao)
    expedicao['operacao'] = 'expedicao'
    recebimento['operacao'] = 'recebimento'

    df_unificado = pd.concat([recebimento, expedicao], ignore_index=True)

    df_unificado['data_hora'] = pd.to_datetime(
        df_unificado['data'].astype(str) + ' ' + df_unificado['hora'].astype(str))

    lista_produtos = lista_produtos.rename(columns={
        'PNC': 'material',
        'Base Bloco': 'base'
    })
    df_unificado = pd.merge(df_unificado, lista_produtos[['material', 'base']],
                           on='material', how='left')

    def calcular_distancia(row):
        origem = row['origem']
        destino = row['destino']

        if origem in matriz_distancias.columns and destino in matriz_distancias.index:
            return matriz_distancias.loc[destino, origem]
        return np.nan

    df_unificado['distancia'] = df_unificado.apply(calcular_distancia, axis=1)

    df_unificado = df_unificado.sort_values('data_hora').reset_index(drop=True)
    colunas_finais = [
        'operacao', 'ordem', 'material', 'base', 'quantidade',
        'origem', 'destino', 'data_hora', 'distancia'
    ]
    df_unificado = df_unificado[colunas_finais]
    df_unificado.to_excel('ordens_unificadas.xlsx', index=False)
    print("Planilha criada")

    return df_unificado

df_final = processar_planilhas()

Planilha unificada criada com sucesso!


In [3]:
import pandas as pd
import numpy as np

matriz_distancias = pd.read_excel('matriz_distancias.xlsx')
sample = pd.read_excel('planilhaDiminuida.xlsx')

In [4]:
sample = sample.dropna()
sample.head()

,operacao,ordem,material,base,quantidade,origem,destino,data_hora,distancia
0,expedicao,2481001,900941452,6,90,AVF0220,e13,2019-11-01 00:01:35,140.2
1,expedicao,2481002,900941452,6,72,AVF0220,e13,2019-11-01 00:01:49,140.2
2,expedicao,2481006,900941452,6,54,AVF0220,e13,2019-11-01 00:06:24,140.2
3,expedicao,2481007,900941452,6,36,AVF0220,e13,2019-11-01 00:06:39,140.2
9,expedicao,2481015,900941452,6,18,AVF0220,e13,2019-11-01 00:28:29,140.2


In [5]:
lista_sample_origem_valores_unicos = sample['origem'].unique()
lista_sample_origem_valores_unicos
lista_sample_destino_valores_unicos = sample['destino'].unique()
lista_sample_destino_valores_unicos
#lista_sample_valores_unicos = lista_sample_origem_valores_unicos + lista_sample_destino_valores_unicos
lista_sample_destino_valores_unicos = list(lista_sample_destino_valores_unicos)
lista_sample_origem_valores_unicos = list(lista_sample_origem_valores_unicos)
lista_sample_valores_unicos = lista_sample_destino_valores_unicos + lista_sample_origem_valores_unicos
lista_sample_valores_unicos

['e13',
 'AVG0105',
 'AVA0121',
 'AVC0202',
 'AVD0116',
 'AVC0218',
 'AVD0107',
 'AVB0215',
 'AVB0207',
 'AVB0202',
 'AVC0113',
 'AVH0105',
 'AVF0108',
 'AVG0202',
 'AVC0115',
 'AVH0218',
 'AVB0224',
 'AVC0208',
 'AVH0103',
 'AVH0216',
 'AVD0109',
 'AVB0220',
 'AVG0218',
 'AVB0107',
 'AVD0202',
 'AVC0222',
 'AVF0107',
 'AVA0209',
 'AVB0208',
 'AVG0102',
 'AVE0119',
 'AVB0128',
 'AVA0219',
 'AVA0231',
 'AVB0110',
 'AVF0130',
 'AVB0106',
 'AVG0216',
 'AVG0201',
 'AVG0108',
 'AVD0209',
 'AVF0120',
 'AVB0105',
 'AVC0116',
 'AVG0123',
 'AVB0118',
 'AVF0220',
 'Esteira02',
 'Esteira03']

In [6]:
matriz_distancias = pd.read_excel('matriz_distancias.xlsx', index_col=0)

submatriz = matriz_distancias.loc[lista_sample_valores_unicos, lista_sample_valores_unicos]
submatriz
submatriz.to_excel('matriz_filtrada.xlsx')

In [9]:
submatriz
sample.to_excel('sample.xlsx')

In [7]:
ordens = pd.read_excel('ordens_unificadas 1.xlsx')
ordens = ordens.dropna()
ordens.to_excel('ordens_unificadas.xlsx', index=False)

In [9]:
from google.colab import files
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import io

# =============================================
# 1. CARREGAMENTO DE ARQUIVOS NO COLAB
# =============================================

# Função para carregar arquivos
def carregar_arquivo_colab(mensagem):
    uploaded = files.upload()
    nome_arquivo = next(iter(uploaded))
    return pd.read_excel(io.BytesIO(uploaded[nome_arquivo]))

print("Carregue o arquivo de ORDENS:")
ordens = carregar_arquivo_colab("sample.xlsx")

print("\nCarregue o arquivo da MATRIZ DE DISTÂNCIAS:")
matriz_dist = carregar_arquivo_colab("matriz_filtrada.xlsx")

# =============================================
# 2. CLASSE OTIMIZADORA (VERSÃO COLAB)
# =============================================

class OtimizadorColab:
    def __init__(self, num_empilhadeiras):
        self.num_empilhadeiras = num_empilhadeiras
        self.resetar()

    def resetar(self):
        self.empilhadeiras = {
            i: {
                'posicao': None,
                'livre_em': datetime.now(),
                'distancia_total': 0.0,
                'ordens_atendidas': []
            } for i in range(self.num_empilhadeiras)
        }
        self.ordens_nao_atendidas = []

    def otimizar(self, ordens, matriz_dist):
        self.resetar()

        # Pré-processamento crítico
        ordens['data_hora'] = pd.to_datetime(ordens['data_hora'], errors='coerce')
        ordens = ordens.dropna(subset=['data_hora']).sort_values('data_hora')

        matriz_dist = matriz_dist.set_index(matriz_dist.columns[0])
        matriz_dist = matriz_dist.applymap(lambda x: float(str(x).replace(',', '.')))

        for _, ordem in ordens.iterrows():
            self._processar_ordem(ordem, matriz_dist)

        return self._gerar_resultados(matriz_dist)

    def _processar_ordem(self, ordem, matriz_dist):
        melhor_emp = None
        melhor_custo = float('inf')

        for emp_id, emp in self.empilhadeiras.items():
            try:
                # Cálculo da distância
                pos_atual = emp['posicao'] or ordem['origem']
                dist = matriz_dist.loc[pos_atual, ordem['origem']] + \
                       matriz_dist.loc[ordem['origem'], ordem['destino']]

                # Cálculo do custo
                tempo_espera = max(0, (ordem['data_hora'] - emp['livre_em']).total_seconds())
                custo = dist + (tempo_espera * 0.1)

                if custo < melhor_custo:
                    melhor_custo = custo
                    melhor_emp = emp_id

            except Exception as e:
                print(f"Erro na ordem {ordem.get('ordem', '?')}: {str(e)}")
                continue

        if melhor_emp is not None:
            self._atribuir_ordem(melhor_emp, ordem, matriz_dist)
        else:
            self.ordens_nao_atendidas.append(ordem.to_dict())

    def _atribuir_ordem(self, emp_id, ordem, matriz_dist):
        emp = self.empilhadeiras[emp_id]
        pos_atual = emp['posicao'] or ordem['origem']

        dist_total = matriz_dist.loc[pos_atual, ordem['origem']] + \
                     matriz_dist.loc[ordem['origem'], ordem['destino']]

        self.empilhadeiras[emp_id] = {
            'posicao': ordem['destino'],
            'livre_em': max(emp['livre_em'], ordem['data_hora']) + \
                        timedelta(seconds=dist_total/10),
            'distancia_total': emp['distancia_total'] + dist_total,
            'ordens_atendidas': emp['ordens_atendidas'] + [ordem.to_dict()]
        }

    def _gerar_resultados(self, matriz_dist):
        resultados = []
        for emp_id, emp in self.empilhadeiras.items():
            for ordem in emp['ordens_atendidas']:
                resultados.append({
                    'ordem': ordem['ordem'],
                    'material': ordem['material'],
                    'origem': ordem['origem'],
                    'destino': ordem['destino'],
                    'empilhadeira': emp_id,
                    'distancia': matriz_dist.loc[ordem['origem'], ordem['destino']],
                    'inicio': ordem['data_hora']
                })

        metricas = {
            'total_ordens': len(resultados) + len(self.ordens_nao_atendidas),
            'atendidas': len(resultados),
            'nao_atendidas': len(self.ordens_nao_atendidas),
            'distancia_total': sum(e['distancia_total'] for e in self.empilhadeiras.values())
        }

        return pd.DataFrame(resultados), metricas

# =============================================
# 3. EXECUÇÃO E DOWNLOAD DOS RESULTADOS
# =============================================

# Configuração
NUM_EMPILHADEIRAS = 12

# Processamento
print("\nIniciando otimização...")
otimizador = OtimizadorColab(NUM_EMPILHADEIRAS)
rotas, metricas = otimizador.otimizar(ordens, matriz_dist)

# Resultados
print("\n=== RESUMO ===")
print(f"Ordens processadas: {metricas['total_ordens']}")
print(f"Ordens atendidas: {metricas['atendidas']}")
print(f"Ordens não atendidas: {metricas['nao_atendidas']}")
print(f"Distância total: {metricas['distancia_total']:.2f}m")

# Salvar e disponibilizar resultados
if not rotas.empty:
    rotas.to_excel('resultados_otimizacao.xlsx', index=False)
    files.download('resultados_otimizacao.xlsx')
    print("\nPlanilha com resultados disponibilizada para download!")
else:
    print("\nAVISO: Nenhuma rota foi gerada. Verifique os dados de entrada.")

# Diagnóstico (opcional)
print("\nPrimeiras linhas dos dados processados:")
print(rotas.head())

Carregue o arquivo de ORDENS:


Saving ordens_unificadas(1).xlsx to ordens_unificadas(1) (1).xlsx

Carregue o arquivo da MATRIZ DE DISTÂNCIAS:


Saving matriz_distancias.xlsx to matriz_distancias (2).xlsx

Iniciando otimização...


<ipython-input-9-f20757cd89f2>:51: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  matriz_dist = matriz_dist.applymap(lambda x: float(str(x).replace(',', '.')))



=== RESUMO ===
Ordens processadas: 32412
Ordens atendidas: 32412
Ordens não atendidas: 0
Distância total: 8357560.23m


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Planilha com resultados disponibilizada para download!

Primeiras linhas dos dados processados:
     ordem   material     origem  destino  empilhadeira  distancia  \
0  2481001  900941452    AVF0220      e13             0      140.2   
1  2481047  900941408  Esteira02  AVG0105             0      230.3   
2  2481060  900941408  Esteira02  AVA0121             0      105.3   
3  2481061  900941408  Esteira02  AVA0121             0      105.3   
4  2481062  900941408  Esteira02  AVA0121             0      105.3   

               inicio  
0 2019-11-01 00:01:35  
1 2019-11-01 01:08:42  
2 2019-11-01 01:14:13  
3 2019-11-01 01:14:28  
4 2019-11-01 01:14:49  
